In [3]:
from selenium.webdriver import Chrome
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pymongo
from selenium.common.exceptions import NoSuchElementException
from cost_of_living import *
from functools import reduce
import matplotlib.pyplot as plt
%matplotlib inline
from hidden import debt
from math import cos, sqrt

## Write a function to produce cost-of-living in multiple cities

In [2]:
data = pd.read_excel('data/citydf.xlsx')
data['city'].replace('Washington', "Washington D.C.", inplace=True)

In [4]:
monthly_constants = {
'Cinema, International Release, 1 Seat' : 2,
'Fitness Club, Monthly Fee for 1 Adult' : 2,
'Basic (Electricity, Heating, Cooling, Water, Garbage) for 915 sq ft Apartment' : 1,
'Internet (60 Mbps or More, Unlimited Data, Cable/ADSL)' : 1,
'Gasoline (1 gallon)' : 15,
'Imported Beer (11.2 oz small bottle)' : 10,
'Bottle of Wine (Mid-Range)' : 2,
'Cappuccino (regular)'  : 30,
'Meal, Inexpensive Restaurant' : 8,
'1 Pair of Jeans (Levis 501 Or Similar)' : 1,
'groceries (dollars)': 600
}

def cost_per_city_1B_Center(citydf, monthly_constants):
    for city in citydf.city.unique():
        total = 0
        for item, price in monthly_constants.items():
            if item != 'groceries (dollars)':
                price = float(citydf[(citydf['city']==city) & (citydf['item']==item)].values[0][3]) * price
                total += price
            else:
                total += price
        total += (float(citydf[(citydf['city']==city) & 
                               (citydf['item']=='Apartment (1 bedroom) in City Centre')].values[0][3]))
        result = {
        'city' : city,
        '1_Bed_Center_Total' : total
                  }
        yield result
        
def cost_per_city_3B_Center(citydf, monthly_constants):
    for city in citydf.city.unique():
        total = 0
        for item, price in monthly_constants.items():
            if item != 'groceries (dollars)':
                price = float(citydf[(citydf['city']==city) & (citydf['item']==item)].values[0][3]) * price
                total += price
            else:
                total += price
        total += round((float(citydf[(citydf['city']==city) & 
                               (citydf['item']=='Apartment (3 bedrooms) in City Centre')].values[0][3]) / 3),2)
        result = {
        'city' : city,
        '3_Bed_Center_Total' : total
                  }
        yield result

def cost_per_city_3B_Outside(citydf, monthly_constants):
    for city in citydf.city.unique():
        total = 0
        for item, price in monthly_constants.items():
            if item != 'groceries (dollars)':
                price = float(citydf[(citydf['city']==city) & (citydf['item']==item)].values[0][3]) * price
                total += price
            else:
                total += price
        total += round(((float(citydf[(citydf['city']==city) & 
                               (citydf['item']=='Apartment (3 bedrooms) Outside of Centre')].values[0][3])) / 3),2)
        result = {
        'city' : city,
        '3_Bed_Outside_Total' : total
                  }
        yield result

def cost_per_city_1B_Outside(citydf, monthly_constants):
    for city in citydf.city.unique():
        total = 0
        for item, price in monthly_constants.items():
            if item != 'groceries (dollars)':
                price = float(citydf[(citydf['city']==city) & (citydf['item']==item)].values[0][3]) * price
                total += price
            else:
                total += price
        total += (float(citydf[(citydf['city']==city) & 
                               (citydf['item']=='Apartment (1 bedroom) Outside of Centre')].values[0][3]))
        result = {
        'city' : city,
        '1_Bed_Outside_Total' : total
                  }
        yield result

In [5]:
def find_cost_of_living(data, monthly_constants):
    B1_center = pd.DataFrame(list(cost_per_city_1B_Center(data, monthly_constants)))
    B3_center = pd.DataFrame(list(cost_per_city_3B_Center(data, monthly_constants)))
    B1_Outside = pd.DataFrame(list(cost_per_city_1B_Outside(data, monthly_constants)))
    B3_Outside = pd.DataFrame(list(cost_per_city_3B_Outside(data, monthly_constants)))
    B1_center.set_index('city', inplace=True)
    B3_center.set_index('city', inplace=True)
    B1_Outside.set_index('city', inplace=True)
    B3_Outside.set_index('city', inplace=True)
    dfs = [B1_center, B3_center, B1_Outside, B3_Outside]
    city_matrix = pd.concat(dfs, axis=1)
    city_matrix['city'] = city_matrix.index
    city_matrix.drop(columns='city', inplace=True)
    return city_matrix

In [6]:
find_cost_of_living(data, monthly_constants)

,1_Bed_Center_Total,3_Bed_Center_Total,1_Bed_Outside_Total,3_Bed_Outside_Total
city,,,,
Palo-Alto,4387.48,3127.55,4094.21,2871.99
San-Jose,3714.08,2500.71,3456.40,2415.23
Portland,2864.56,2296.22,2485.55,1980.12
Austin,2969.98,2289.77,2399.66,1943.74
Denver,2907.57,2211.63,2574.14,1982.24
San-Diego,3210.41,2406.10,2804.00,2184.68
Washington D.C.,3517.34,2701.31,3064.00,2314.06
Boston,3821.77,2853.61,3136.72,2316.74
San-Francisco,4806.91,3329.76,4053.38,2852.41


In [17]:
cities = {'New-York': 'New York, New York', 'San-Francisco' : 'San-Francisco, California', 
          'Honolulu' : 'Honolulu, Hawaii', 'Arlington' : 'Arlington, Virgina', 'Anchorage' : 'Anchorage, Alaska',
          'Brooklyn' : 'Brooklyn, New York', 'Santa-Rosa' : 'Santa-Rosa, California', 'Washington' : 'Washington, D.C.',
          'Charleston' : 'Charleston, South Carolina', 'Berkeley' : 'Berkeley, California', 
          'Seattle' : 'Seattle, Washington', 'Jersey-City': 'Jersey City, New Jersey', 
          'Boston' : 'Boston, Massachusetts', 'Oakland' : 'Oakland, California', 'Hartford' : 'Hartford, Connecticut',
          'Philadelphia' : 'Philadelphia, Pennsylvania', 'Albany' : 'Albany, New York','Long-Beach' : 'Long Beach, California', 
          'Irvine' : 'Irvine, California', 'Miami': 'Miami, Florida', 'Los-Angeles' : 'Los Angeles, California',
          'Boulder' : 'Boulder, Colorado', 'Sacramento' : 'Sacramento, California', 'Chicago' : 'Chicago, Illinois',
          'Greenville' : 'Greenville, South Carolina', 'Burlington' : 'Burlington, Vermont', 'San-Jose' : 'San Jose, California',
          'Everett' : 'Everett, Washington', 'Birmingham' : 'Birmingham, Alabama', 'Pittsburgh' : 'Pittsburgh, Pennsylvania',
          'Minneapolis' : 'Minneapolis, Minnesota', 'Baltimore' : 'Baltimore, Maryland', 'Portland' : 'Portland, Oregon',
          'Rochester' : 'Rochester, New York', 'Charlotte' : 'Charlotte, North Carolina',
          'Santa-Barbara' : 'Santa Barbara, California', 'West-Palm-Beach' : 'West Palm Beach, Florida', 
          'Tacoma' : 'Tacoma, Washington', 'Portland-ME' : 'Portland, Maine', 'Bellingham' : 'Bellingham, Washington',
          'San-Diego' : 'San Diego, California', 'Saint-Paul' : 'Saint Paul, Minnesota', 'Atlanta' : 'Atlanta, Georgia',
          'Asheville' : 'Asheville, North Carolina', 'Eugene' : 'Eugene, Oregon', 'Saint-Petersburg' : 'Saint Petersburg, Florida',
          'Buffalo' : 'Buffalo, New York', 'Peoria' : 'Peoria, Illinois', 'Austin' : 'Austin, Texas',
          'Cleveland' : 'Cleveland, Ohio', 'Jacksonville' : 'Jacksonville, Florida', 'Milwaukee' : 'Milwaukee, Wisconsin',
          'Denver' : 'Denver, Colorado', 'Indianapolis' : 'Indianapolis, Indiana', 'Arlington' : 'Arlington, Texas',
          'Ann-Arbor' : 'Ann Arbor, Michigan', 'Raleigh' : 'Raleigh, North Carolina', 'Orlando' : 'Orlando, Florida',
          'Tampa' : 'Tampa, Florida', 'Saint-Louis' : 'Saint Louis, Missouri', 'Nashville' : 'Nashville, Tennessee',
          'Syracuse' : 'Syracuse, New York', 'Salem' : 'Salem, Oregon', 'Kansas-City' : 'Kansas City, Missouri',
          'Spokane' : 'Spokane, Washington', 'Bakersfield' : 'Bakersfield, California', 'Columbus' : 'Columbus, Ohio',
          'Tallahassee' : 'Tallahassee, Florida', 'Memphis' : 'Memphis, Tennessee', 'Las-Vegas' : 'Las Vegas, Nevada',
          'Columbia' : 'Columbia, South Carolina', 'Dallas' : 'Dallas, Texas', 'Detroit': 'Detroit, Michigan', 
          'Phoenix' : 'Phoenix, Arizona', 'Greensboro' : 'Greensboro, North Carolina', 'Richmond' : 'Richmond, Virgina',
          'Cincinnati' : 'Cincinnati, Ohio', 'Fort Worth' : 'Fort Worth, Texas', 'Fresno' : 'Fresno, California',
          'Grand Rapids' : 'Grand Rapids, Michigan', 'Tucson' : 'Tucson, Arizona', 'Fremont' : 'Fremont, California',
          'Riverside' : 'Riverside, California', 'Albuquerque' : 'Albuquerque, New Mexico', 'Tulsa' : 'Tulsa, Oklahoma',
          'Chattanooga' : 'Chattanooga, Tennessee', 'Louisville' : 'Louisville, Kentucky', 'Salt-Lake-City' : 'Salt Lake City, Utah',
          'Houston' : 'Houston, Texas', 'Oklahoma City' : 'Oklahoma-City, Oklahoma', 'Des-Moines' : 'Des Moines, Iowa',
          'Fayetteville' : 'Fayetteville, Arizona', 'Madison' : 'Madison, Wisconsin', 'Wichita' : 'Wichita, Kansas',
          'San-Antonio' : 'San Antonio, Texas', 'Knoxville' : 'Knoxville, Tennessee', 'Vancouver' : 'Vancouver, Washington',
          'Toledo' : 'Toledo, Ohio', 'Boise' : 'Boise, Idaho', 'Reno' : 'Reno, Nevada', 'Little-Rock' : 'Little Rock, Arizona',
          'Huntsville' : 'Huntsville, Alabama', 'El-Paso' : 'El Paso, Texas', 'Lexington' : 'Lexington, Kentucky'}


In [20]:
city_data = list(scrape_cities(cities.keys()))

In [21]:
citydf = pd.DataFrame(city_data)

In [31]:
def get_city_data(city):
    city_data = pd.DataFrame(list(scrape_cities(city)))
    return city_data

In [39]:
hoboken_data = get_city_data(['Hoboken'])

## Write function to standardize prices

#### Start by finding a city that has missing values. For this example, we'll use Hoboken. We'll then concatenate Hoboken to our other data.

In [84]:
frames = [citydf, hoboken_data]
result = pd.concat(frames)

#### Now we'll transpose the DF and move the columns around so that we can work with them.

In [106]:
import string

In [88]:
play = result

In [80]:
play.columns = play.iloc[1]

In [77]:
play.set_index('city', inplace=True)

In [93]:
play.replace('?', np.NAN, inplace=True)

In [108]:
play.dropna(inplace=True)
play['price'] = play['price'].apply(clean_number)

In [110]:
art = pd.crosstab(index=play['city'], columns=play['item'], values=play['price'], aggfunc=np.mean)

In [116]:
art.loc[art.isnull().sum(axis=1).astype(np.bool)]

item,1 Pair of Jeans (Levis 501 Or Similar),1 Pair of Men Leather Business Shoes,1 Pair of Nike Running Shoes (Mid-Range),"1 Summer Dress in a Chain Store (Zara, H&M, ...)",1 min. of Prepaid Mobile Tariff Local (No Discounts or Plans),Apartment (1 bedroom) Outside of Centre,Apartment (1 bedroom) in City Centre,Apartment (3 bedrooms) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apples (1 lb),...,"Rice (white), (1 lb)",Taxi 1 mile (Normal Tariff),Taxi 1hour Waiting (Normal Tariff),Taxi Start (Normal Tariff),Tennis Court Rent (1 Hour on Weekend),Tomato (1 lb),Toyota Corolla 1.6l 97kW Comfort (Or Equivalent New Car),Volkswagen Golf 1.4 90 KW Trendline (Or Equivalent New Car),Water (1.5 liter bottle),Water (11.2 oz small bottle)
city,,,,,,,,,,,,,,,,,,,,,
Hoboken,80.0,100.0,96.67,50.0,0.1,1875.0,2233.33,4500.0,4800.0,3.76,...,4.5,5.0,NaN,6.0,NaN,4.5,18500.0,20000.0,2.65,1.67


In [107]:
def clean_number(text):
    """Remove invalid characters from text representation of number and cast to float"""
    valid_chars = ""
    for char in text.strip():
        if char in string.digits + '.':
            valid_chars += char
    return float(valid_chars)

In [117]:
art

item,1 Pair of Jeans (Levis 501 Or Similar),1 Pair of Men Leather Business Shoes,1 Pair of Nike Running Shoes (Mid-Range),"1 Summer Dress in a Chain Store (Zara, H&M, ...)",1 min. of Prepaid Mobile Tariff Local (No Discounts or Plans),Apartment (1 bedroom) Outside of Centre,Apartment (1 bedroom) in City Centre,Apartment (3 bedrooms) Outside of Centre,Apartment (3 bedrooms) in City Centre,Apples (1 lb),...,"Rice (white), (1 lb)",Taxi 1 mile (Normal Tariff),Taxi 1hour Waiting (Normal Tariff),Taxi Start (Normal Tariff),Tennis Court Rent (1 Hour on Weekend),Tomato (1 lb),Toyota Corolla 1.6l 97kW Comfort (Or Equivalent New Car),Volkswagen Golf 1.4 90 KW Trendline (Or Equivalent New Car),Water (1.5 liter bottle),Water (11.2 oz small bottle)
city,,,,,,,,,,,,,,,,,,,,,
Albany,46.09,96.90,80.82,35.80,0.10,816.27,931.25,1358.33,1554.09,2.70,...,1.88,5.00,30.00,7.25,32.50,2.99,21000.00,23000.0,1.92,1.78
Albuquerque,39.97,81.73,69.46,42.50,0.10,708.85,830.00,1161.33,1329.69,1.82,...,1.56,2.55,28.40,2.98,1.67,2.07,20808.33,21725.0,2.19,1.48
Anchorage,42.91,103.17,77.36,45.33,0.10,981.67,1247.06,1708.53,1945.88,3.13,...,3.20,2.50,50.00,2.75,25.67,2.08,21529.50,22500.0,2.17,1.86
Ann-Arbor,42.38,94.28,82.50,33.57,0.10,976.27,1411.11,1814.29,2785.71,1.89,...,1.36,3.00,24.00,4.75,16.33,1.65,21666.67,25107.5,1.75,1.22
Arlington,43.56,105.80,76.44,45.83,0.10,1670.60,2107.85,2833.33,3514.29,2.50,...,3.23,2.10,23.50,3.50,21.67,1.50,18430.33,21455.0,1.75,1.75
Asheville,47.33,111.50,77.50,37.00,0.10,888.24,1146.67,1469.23,1987.50,2.55,...,2.44,5.00,18.50,3.25,17.50,1.85,24333.33,23500.0,1.75,1.46
Atlanta,40.06,87.07,75.97,36.75,0.10,1047.10,1506.57,1662.12,2703.85,2.29,...,1.72,2.00,21.00,2.50,16.17,1.50,19694.44,22000.0,1.82,1.60
Austin,42.64,98.73,75.44,36.78,0.10,1057.25,1625.06,1782.07,2830.71,2.15,...,1.30,2.70,29.00,2.54,18.33,1.55,20489.37,20910.0,1.45,1.69
Bakersfield,41.81,93.57,76.88,40.00,0.11,747.27,863.64,1294.62,1375.56,1.70,...,1.62,2.75,27.00,3.00,3.33,1.23,22800.00,25000.0,1.70,1.30


In [120]:
art.shape

(101, 54)